In [181]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [182]:
# Constants

gammac = 1.4
gammah = 1.33

cpa = 1.005
cp = 1.148
R = 287

In [183]:
# Compressor Values

#Ca4 = 226.4117 # m/s
Ca4 = 270
Ca5 = Ca4
N = 208.05 # Rev/s
deltaTocomp = 439.58 # K

In [184]:
To4 = 2275 # K
polyt = 0.92
#flowcoeff Greater than 0.78
#loadcoeff Cannot Exceed 3.3
mfahot = 40 # kg/s
mfthot = mfahot + (mfahot*.02)

In [185]:
# Turbine PARAMETERS

htratioturbine = 0.74

In [186]:
deltaToturb = (deltaTocomp*cpa*0.99)/cp

T4 = To4-((Ca4**2)/(2*cp*1000))

po4 = 15 # bar
p4 = po4*(T4/To4)**(gammah/(gammah-1))

density4 = (p4*100000)/(R*T4)

print(density4)


2.2015640010505257


In [187]:
A4 = mfthot/(density4*Ca4)

M4 = Ca4/(np.sqrt(1.333*R*To4))

print("Turbine Inlet Area = %3.4f m^2" % (A4))

print("Turbine Inlet Mach Number = %3.2f" % (M4))

Turbine Inlet Area = 0.0686 m^2
Turbine Inlet Mach Number = 0.29


In [188]:
r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

r4h = htratioturbine*r4t

r4m = (r4t+r4h)/2

print("r4t = %3.3f m" % (r4t))
print("rm = %3.3f m" % (r4m))
print("r4h = %3.3f m" % (r4h))

r4t = 0.220 m
rm = 0.191 m
r4h = 0.163 m


In [189]:
To5 = To4 - deltaToturb

T5 = To5 - (Ca5**2)/(2*cp*1000)

po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

p5 = po5*(T5/To5)**(gammah/(gammah-1))

density5 = (p5*100000)/(R*T5)

A5 = mfthot/(density5*Ca5)

print("Turbine Exhaust Area = %3.4f m^2" % (A5))

Turbine Exhaust Area = 0.1137 m^2


In [190]:
# constant mean radius
#bladeheight5 = A5/(2*np.pi*rm)
#
#r5t = rm+(bladeheight5/2)
#
#r5h = rm-(bladeheight5/2)


# constant hub radius
r5h = r4h
r5t = np.sqrt(A5/np.pi + (r5h)**2)
r5m = (r5t+r5h)/2

print("r5t = %3.3f m" % (r5t))
print("r5m = %3.3f m" % (r5m))
print("r5h = %3.3f m" % (r5h))

r5t = 0.250 m
r5m = 0.206 m
r5h = 0.163 m


In [191]:
U5h = r5h*(2*np.pi*N)

print(U5h)

flow5 = Ca5/U5h

print(flow5)

bladeloading = 3.3

print(bladeloading)

deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

print(deltaTos)

stageest = deltaToturb/deltaTos

print("Estimation of %3.1f Stages" % (stageest))

212.58157265609796
1.2701006800659538
3.3
64.95211350553203
Estimation of 5.9 Stages


In [192]:
U4t = 2*np.pi*r4t*N
U4m = 2*np.pi*r4m*N
U4h = 2*np.pi*r4h*N

In [193]:
stageactual = 5
# Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
stagevals = np.zeros((stageactual+1,3,17))
stagevals[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca4,0,0,0,0,0,po4,0,0]])

$$
\psi = \frac{2 c_p \Delta T_{0s}}{U^2} \\
\phi = \frac{C_a}{U} \\
tan \beta_2 = \frac{1}{2 \phi} (\frac{\psi}{2} - 2\Lambda) \\
tan \beta_3 = \frac{1}{2 \phi} (\frac{\psi}{2} + 2\Lambda) \\
$$

In [194]:
def returnstage(deltaTo,U,Ca,DOR):
    bladeloading = (2*cp*1000*deltaTo)/(U**2)
    flow = Ca/U
    beta2 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)-(2*DOR))))
    beta3 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)+(2*DOR))))
    alpha2 = np.degrees(np.arctan(np.tan(np.radians(beta2))+(1/flow)))
    alpha3 = np.degrees(np.arctan((np.tan(np.radians(beta3))-(1/flow))))

    Cw2 = Ca*np.tan(np.radians(alpha2))
    Cw3 = Ca*np.tan(np.radians(alpha3))

    V2 = Ca/(np.cos(np.radians(beta2)))
    V3 = Ca/(np.cos(np.radians(beta3)))

    return beta2,beta3,alpha2,alpha3,Cw2,Cw3,V2,V3,bladeloading,flow

In [195]:
i = 1

deltaTosact = deltaToturb/stageactual

print(deltaTosact)

Cw1t = 0 # m/s
Cw1m = 0 # m/s
Cw1h = 0 # m/s

# This runs through the params for each stage and saves them to an array
while i < stageactual+1:
    if i == 0:
        DOR = 0.5
    if i == 1:
        DOR = 0.55
    if i == 2:
        DOR = 0.6
    if i == 3:
        DOR = 0.7
    else:
        DOR = 0.8

    rtstage = ((r5t-r4t)/(stageactual-1))*i+r4t 
    rmstage = ((r5m-r4m)/(stageactual-1))*i+r4m
    rhstage = ((r5h-r4h)/(stageactual-1))*i+r4h

    Utstage = 2*np.pi*rtstage*N
    Umstage = 2*np.pi*rmstage*N
    Uhstage = 2*np.pi*rhstage*N

    stageTo3 = stagevals[i-1,0,6] + deltaTosact

    # I PROBABLY NEED CHANGED
    pr = (1+((polyt*deltaTosact)/(stagevals[i-1,0,6])))**(gammac/(gammac-1))

    # DO NOT TOUCH THE COMMENTED FUNCTION UNLESS THE FIRST STAGE VALUES ARE FUCKED UP

    """if i == 1:
        deltawhirlt = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Utstage))
        Cw2t = deltawhirlt - Cw1t
        stagebeta1tip = np.degrees(np.arctan(Utstage/Castage))
        stagebeta2tip = np.degrees(np.arctan((Utstage-Cw2t)/Castage))
        stagealpha1tip = 0
        stagealpha2tip = np.degrees(np.arctan((Cw2t)/Castage))

        V1t = Castage/(np.cos(np.radians(stagebeta1tip)))
        dehallert = np.cos(np.radians(stagebeta1tip))/np.cos(np.radians(stagebeta2tip))
        V2t = V1t*dehallert

        deltawhirlm = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Umcomp))
        Cw2m = deltawhirlm - Cw1m
        stagebeta1mean = np.degrees(np.arctan(Umcomp/Castage))
        stagebeta2mean = np.degrees(np.arctan((Umcomp-Cw2m)/Castage))
        stagealpha1mean = 0
        stagealpha2mean = np.degrees(np.arctan((Cw2m)/Castage))

        V1m = Castage/(np.cos(np.radians(stagebeta1mean)))
        dehallerm = np.cos(np.radians(stagebeta1mean))/np.cos(np.radians(stagebeta2mean))
        V2m = V1m*dehallerm

        deltawhirlh = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Uhcomp))
        Cw2h = deltawhirlh - Cw1h
        stagebeta1hub = np.degrees(np.arctan(Uhcomp/Castage))
        stagebeta2hub = np.degrees(np.arctan((Uhcomp-Cw2h)/Castage))
        stagealpha1hub = 0
        stagealpha2hub = np.degrees(np.arctan((Cw2h)/Castage))

        V1h = Castage/(np.cos(np.radians(stagebeta1hub)))
        dehallerh = np.cos(np.radians(stagebeta1hub))/np.cos(np.radians(stagebeta2hub))
        V2h = V1h*dehallerh
    else: """
    stagebeta2tip,stagebeta3tip,stagealpha2tip,stagealpha3tip,Cw2t,Cw3t,V2t,V3t,bladelt,flowt = returnstage(deltaTosact,Utstage,Ca4,DOR)

    stagebeta2mean,stagebeta3mean,stagealpha2mean,stagealpha3mean,Cw2m,Cw3m,V2m,V3m,bladelm,flowm = returnstage(deltaTosact,Umstage,Ca4,DOR)

    stagebeta2hub,stagebeta3hub,stagealpha2hub,stagealpha3hub,Cw2h,Cw3h,V2h,V3h,bladelh,flowh = returnstage(deltaTosact,Uhstage,Ca4,DOR)

    #pr = (1+((polyc*(((Umstage*Castage)/(cpa*1000))*(np.tan(np.radians(stagebeta1mean))-np.tan(np.radians(stagebeta2mean)))))/(stagevals[i-1,0,6])))**(gammac/(gammac-1))
    stagepo3 = stagevals[i-1,0,15]*pr

    stagevals[i,:,:] = np.array([[stagealpha2hub,stagealpha3hub,stagebeta2hub,stagebeta3hub,rhstage,deltaTosact,stageTo3,Uhstage,Ca4,Cw2h,Cw3h,V2h,V3h,pr,stagepo3,bladelh,flowh],[stagealpha2mean,stagealpha3mean,stagebeta2mean,stagebeta3mean,rmstage,deltaTosact,stageTo3,Umstage,Ca4,Cw2m,Cw3m,V2m,V3m,pr,stagepo3,bladelm,flowm],[stagealpha2tip,stagealpha3tip,stagebeta2tip,stagebeta3tip,rtstage,deltaTosact,stageTo3,Utstage,Ca4,Cw2t,Cw3t,V2t,V3t,pr,stagepo3,bladelt,flowt]])

    i += 1

76.19514303135888


In [196]:
def valtable(location,vals):
    if location == 0:
        print("Hub Values:")
    elif location == 1:
        print("Mean Values:")
    elif location == 2:
        print("Tip Values:")
    
    num = 0
    stages = ["Inlet"]

    while num < stageactual:
        stages.append("Stage "+str(num+1))
        num += 1

    pd.set_option("display.precision", 4)
    tabletry = pd.DataFrame({"Alpha 1": vals[:,location,0],"Alpha 2": vals[:,location,1],"Beta 1": vals[:,location,2],"Beta 2": vals[:,location,3],"Radius": vals[:,location,4],"Delta To": vals[:,location,5],"To2": vals[:,location,6],"U": vals[:,location,7],"Ca": vals[:,location,8],"Cw1": vals[:,location,9],"Cw2": vals[:,location,10],"V1": vals[:,location,11],"V2": vals[:,location,12],"PR": vals[:,location,13],"po2": vals[:,location,14],"Blade Loading": vals[:,location,15],"Flow": vals[:,location,16]},index=stages)

    return tabletry

In [197]:
valtable(0,stagevals)

Hub Values:


,Alpha 1,Alpha 2,Beta 1,Beta 2,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,212.5816,270.0,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,42.5973,31.1539,7.5263,54.3042,0.1626,76.1951,2351.1951,212.5816,270.0,248.2539,163.2212,272.3463,462.7394,1.1121,0.0000,3.8712,1.2701
Stage 2,42.5973,31.1539,7.5263,54.3042,0.1626,76.1951,2427.3903,212.5816,270.0,248.2539,163.2212,272.3463,462.7394,1.1083,4.2905,3.8712,1.2701
Stage 3,44.9482,27.7349,11.9066,52.7093,0.1626,76.1951,2503.5854,212.5816,270.0,269.5120,141.9631,275.9367,445.6477,1.1048,4.2768,3.8712,1.2701
Stage 4,42.5973,31.1539,7.5263,54.3042,0.1626,76.1951,2579.7806,212.5816,270.0,248.2539,163.2212,272.3463,462.7394,1.1015,4.2641,3.8712,1.2701
Stage 5,42.5973,31.1539,7.5263,54.3042,0.1626,76.1951,2655.9757,212.5816,270.0,248.2539,163.2212,272.3463,462.7394,1.0984,4.2521,3.8712,1.2701


In [198]:
valtable(1,stagevals)

Mean Values:


,Alpha 1,Alpha 2,Beta 1,Beta 2,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,249.9270,270.0,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,39.4978,24.0665,-6.8349,54.2825,0.1950,76.1951,2351.1951,254.9159,270.0,222.5535,120.5872,271.9326,462.4960,1.1121,0.0000,2.6922,1.0592
Stage 2,39.2065,23.3028,-8.3536,54.3329,0.1988,76.1951,2427.3903,259.9049,270.0,220.2580,116.2960,272.8954,463.0628,1.1083,4.2905,2.5898,1.0388
Stage 3,42.1714,17.5982,-4.3035,52.3946,0.2026,76.1951,2503.5854,264.8938,270.0,244.5759,85.6396,270.7634,442.4632,1.1048,4.2768,2.4932,1.0193
Stage 4,38.6640,21.8159,-11.2794,54.4596,0.2065,76.1951,2579.7806,269.8827,270.0,216.0322,108.0791,275.3178,464.4945,1.1015,4.2641,2.4019,1.0004
Stage 5,38.4116,21.0918,-12.6871,54.5347,0.2103,76.1951,2655.9757,274.8717,270.0,214.0886,104.1400,276.7573,465.3491,1.0984,4.2521,2.3155,0.9823


In [199]:
valtable(2,stagevals)

Tip Values:


,Alpha 1,Alpha 2,Beta 1,Beta 2,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,287.2724,270.0,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,37.4207,17.9917,-18.5618,54.9536,0.2274,76.1951,2351.1951,297.2503,270.0,206.5854,87.6853,284.8159,470.1865,1.1121,0.0000,1.9800,0.9083
Stage 2,37.0464,16.6819,-20.9597,55.1765,0.2350,76.1951,2427.3903,307.2281,270.0,203.8024,80.9112,289.1312,472.8130,1.1083,4.2905,1.8534,0.8788
Stage 3,40.7979,8.9903,-17.3135,53.1242,0.2427,76.1951,2503.5854,317.2060,270.0,233.0407,42.7171,282.8141,449.9385,1.1048,4.2768,1.7387,0.8512
Stage 4,36.4068,14.1834,-25.3771,55.6741,0.2503,76.1951,2579.7806,327.1839,270.0,199.1109,68.2373,298.8356,478.8089,1.1015,4.2641,1.6342,0.8252
Stage 5,36.1364,12.9902,-27.4095,55.9439,0.2579,76.1951,2655.9757,337.1618,270.0,197.1505,62.2858,304.1433,482.1394,1.0984,4.2521,1.5389,0.8008
